In [1]:
%load_ext autoreload
%autoreload 2
import boto3
import os

from tools.document_summarizer import document_summarizer, document_summarizer_toolspec
from tools.image_summarizer import image_summarizer, image_summarizer_toolspec
from tools.country_canvas import country_canvas, country_canvas_toolspec
bedrock_runtime_client = boto3.client("bedrock-runtime", region_name='us-east-1')

In [2]:
model_id_micro = "amazon.nova-micro-v1:0"

In [3]:
def document_summarizer(path:str):
    '''
    Tool to summarize a given document i.e. directly send the document to Bedrock without extracting the text in a document
    '''
    with open(path, "rb") as doc_file:
        doc_bytes = doc_file.read()
    
    doc_message = {
    "role": "user",
    "content": [
        {
            "document": {
                "name": "document",
                "format": "pdf",
                "source": {
                    "bytes": doc_bytes #no base64 encoding required!
                }
            }
        },
        { "text": "For a given document, give me the summary in the bulleted format. Summary generated should be formatted with markdown. Include images and tables as well in markdown" }
    ]
}
    response = bedrock_runtime_client.converse(
    modelId=model_id_micro,
    messages=[doc_message],
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0
    },
)
    response_text = response['output']['message']['content'][0]['text']
    print(response_text)
    return

In [5]:
SYSTEM_PROMPT = """
You are an helpful AI assistant that provides for user-specified query using only
the Document_Summarzier_Tool or Architecture_Summazier_Tool or ,ImageGenerator_Tool which expects a query from the user. 
To use the tool, you strictly apply the provided tool specification.

- Explain your step-by-step process, and give brief updates before each step.
- Use the Document_Summazier_Tool for any summarization queries, Image_Summazier_Tool for Architecture image related queries,Canvas_Tool for given country..
- If only country name is mentioned, use ImageGenerator_Tool for image generation.
- If .png present in the query use Image_Summazier_Tool for architecture description.
- If .pdf present in the query use Document_Summazier_Tool for summary generation
- Repeat the tool use for subsequent requests if necessary.
- If the tool errors, apologize, explain information is unavailable, and suggest other options.
- Only respond to generic queries. Remind off-topic users of your purpose. 
- Complete the entire process until you have all required data before sending the complete response.
"""

In [6]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/aws-genai/agentic-workflows/AmazonNova/agents'

In [7]:
#
# Step 4: Define the initial message and conversation
#
conversation = []
initial_message = {
    "role": "user",
    "content": [
        { "text": "Generate summary for document at location aws-genai/AmazonNova/data/Intro_DVC_Tool.pdf" }
    ],
}
conversation.append(initial_message)

In [10]:
conversation = []
initial_message = {
    "role": "user",
    "content": [
        { "text": "/home/ec2-user/SageMaker/aws-genai/agentic-workflows/AmazonNova/data/Complex-AWS-Architecture.png" }
    ],
}
conversation.append(initial_message)

In [11]:
#
# Step 5: Send the message with the tool config and system prompt
#
SUMMARIZER_SPEC_TOOL = document_summarizer_toolspec()
IMAGE_SUMMARIZER_SPEC_TOOL = image_summarizer_toolspec()
COUNTRY_CANVAS_SPEC_TOOL = country_canvas_toolspec()
response = bedrock_runtime_client.converse(
    modelId=model_id_micro,
    messages=conversation,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0
    },
    toolConfig={
            "tools": [SUMMARIZER_SPEC_TOOL, IMAGE_SUMMARIZER_SPEC_TOOL, COUNTRY_CANVAS_SPEC_TOOL]
    },
    system=[{"text": SYSTEM_PROMPT}]
)

In [12]:
response

{'ResponseMetadata': {'RequestId': '11a23380-c219-4c45-abad-342e7e3277f6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 17 Feb 2025 18:22:18 GMT',
   'content-type': 'application/json',
   'content-length': '657',
   'connection': 'keep-alive',
   'x-amzn-requestid': '11a23380-c219-4c45-abad-342e7e3277f6'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': '<thinking>The user has provided a path to an image file, specifically "Complex-AWS-Architecture.png". Since the file extension is .png, I will use the Image_Summazier_Tool to generate a summary description of the architecture depicted in the image.</thinking>\n\n'},
    {'toolUse': {'toolUseId': 'tooluse_JEPw5KZFS9uBjwz_1uGOPA',
      'name': 'Image_Summazier_Tool',
      'input': {'query': '/home/ec2-user/SageMaker/aws-genai/agentic-workflows/AmazonNova/data/Complex-AWS-Architecture.png'}}}]}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 792, 'outputTokens': 218, 'total

In [13]:
response_message = response['output']['message']
conversation.append(response_message)
response_content_blocks = response_message['content']

follow_up_content_blocks = []

for content_block in response_content_blocks:
    if 'text' in content_block:
      print(content_block['text'])
    elif 'toolUse' in content_block:
      tool_use_block = content_block['toolUse']
      tool_use_name = tool_use_block['name']

      if tool_use_name == 'Document_Summazier_Tool':
        print(tool_use_block['input']['query'])
        tool_result_value = document_summarizer(tool_use_block['input']['query'])
        ## for follow up queries
        follow_up_content_blocks.append({
            "toolResult": {
                "toolUseId": tool_use_block['toolUseId'],
                "content": [
                    {
                        "json": {
                            "result": tool_result_value
                        }
                    }
                ]
            }
        })
      elif tool_use_name == 'Image_Summazier_Tool':
        image_summarizer(tool_use_block['input']['query'])
        
        ## for follow up queries
        follow_up_content_blocks.append({
            "toolResult": {
                "toolUseId": tool_use_block['toolUseId'],
                "content": [
                    {
                        "json": {
                            "result": tool_result_value
                        }
                    }
                ]
            }
        })
        
      elif tool_use_name == 'Canvas_Tool':
        country_canvas(tool_use_block['input'])
        
        ## for follow up queries
        follow_up_content_blocks.append({
            "toolResult": {
                "toolUseId": tool_use_block['toolUseId'],
                "content": [
                    {
                        "json": {
                            "result": tool_result_value
                        }
                    }
                ]
            }
        })
     
      # Handle unknown tools
      else:
        follow_up_content_blocks.append({
          "toolResult": {
            "toolUseId": tool_use_block['toolUseId'],
            "content": [
                {
                    "text": "unknown tool" + tool_use_name
              
                }
            ],
            "status": "error"
          }
        })

<thinking>The user has provided a path to an image file, specifically "Complex-AWS-Architecture.png". Since the file extension is .png, I will use the Image_Summazier_Tool to generate a summary description of the architecture depicted in the image.</thinking>


The architecture diagram depicts a comprehensive workflow for a web application hosted on AWS, involving various AWS services for different functionalities. Here's a breakdown of the workflow and its benefits:

Workflow Breakdown:

Web Console:

Amazon CloudFront: Serves as a CDN to deliver content with low latency and high transfer speeds to users globally.
Amazon S3: Stores static web content like HTML, CSS, JavaScript, and images.

REST API:

Amazon API Gateway: Manages and secures RESTful APIs, acting as a front door for applications to access data, business logic, or functionality from backend services.
AWS Lambda: Executes code in response to triggers such as changes to data or system state, enabling serverless computing.
